<a href="https://colab.research.google.com/github/steliosg23/PDS-A2/blob/main/Finetuning_PubMedBERT_PDS_A2_FHD_Benchmarks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignement 2
### Food Hazard Detection

# Benchmarks - Advanced Model: PubMedBERT

In this task, we aim to classify food safety-related incidents based on two distinct types of input data: short texts (title) and long texts (text).

Using Advanced Model: PubMedBERT  


For each of these input types, we perform the following two subtasks:

**Subtasks (Performed Separately for  title and text):**

**Subtask 1:**

- Classify hazard-category (general hazard type).

- Classify product-category (general product type).

**Subtask 2:**

- Classify hazard (specific hazard).
- Classify product (specific product).

We use all features (year, month, day, country, and the text feature) as input.

Thus, we treat title and text as two distinct data sources, with each undergoing its own preprocessing, model training, and evaluation for all four targets.

# Mount Google Drive and Load Dataset

In this cell, we perform the following steps:
1. **Mount Google Drive**: We mount Google Drive to access files stored there, making it accessible within Google Colab.
2. **Define File Path**: We define the path to the `incidents_train.csv` file stored in Google Drive. This path will be used to load the dataset.
3. **Load Dataset**: We use `pandas` to read the CSV file into a DataFrame and then drop the unnecessary column (`Unnamed: 0`) to clean the data.

This process is essential for loading and preparing the dataset for further analysis or training.


In [23]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the file on Google Drive
train_path = '/content/drive/MyDrive/Data/incidents_train.csv'

# Load the dataset
df = pd.read_csv(train_path)
df = df.drop(columns=['Unnamed: 0'])

Mounted at /content/drive


# Import Required Libraries

This cell imports all the necessary libraries for data processing, model training, evaluation, and visualization.
- `pandas` for data manipulation.
- `re` for regular expressions to clean text.
- `train_test_split` from `sklearn.model_selection` to split the data into training and testing sets.
- `f1_score` and `classification_report` from `sklearn.metrics` to evaluate the model's performance.
- `torch` and `transformers` for handling deep learning models and tokenization.
- `tqdm` for showing progress bars during training.
- `matplotlib` for visualizing the results.


In [8]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm  # Import tqdm for progress bars
import matplotlib.pyplot as plt  # Import matplotlib for plotting


# Configure Hyperparameters

This cell sets up the hyperparameters for model training, including:
- `max_len`: The maximum length of the input sequences.
- `batch_size`: The number of samples per batch.
- `learning_rate`: The learning rate for the optimizer.
- `epochs`: The number of training epochs.
- `model_name`: The pre-trained model (PubMedBERT in this case) to use for fine-tuning.


In [9]:
# Hyperparameters configuration
config = {
    'max_len': 128,
    'batch_size': 16,
    'learning_rate': 1e-5,
    'epochs': 1,
    'model_name': "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"
}


# Set Device for Training

This cell checks whether a GPU is available and sets the device for training (either `cuda` for GPU or `cpu` for CPU).
It prints the device being used for training.


In [10]:
# Set device for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cpu


# Define Custom Dataset Class

Here we define a custom PyTorch `Dataset` class to handle the text data. This class takes in the input texts, labels, tokenizer, and maximum sequence length, and implements methods to return tokenized inputs and labels in a batch.
- The `__len__` method returns the number of samples in the dataset.
- The `__getitem__` method returns tokenized input data (input ids and attention mask) and the corresponding label for a given index.


In [11]:
# Custom Dataset for Text Data
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


# Text Preprocessing - Cleaning Function

This function cleans the text by removing any non-alphanumeric characters (e.g., punctuation) and converts the text to lowercase.
This helps standardize the text for further processing, such as tokenization and model input.


In [12]:
# Function to clean text (title or text) and remove stopwords
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.lower()
    text = ' '.join(text.split())
    return text


# Preprocessing the Text Data

In this step, we apply the `clean_text` function to the `title` and `text` columns of the DataFrame to clean and preprocess the text data. This ensures that all the text data used for model input is in a consistent format.


In [13]:
# Load tokenizer for Microsoft PubMedBERT model
tokenizer = AutoTokenizer.from_pretrained(config['model_name'])

# Assuming df is your DataFrame
df['title'] = df['title'].apply(clean_text)
df['text'] = df['text'].apply(clean_text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

NameError: name 'df' is not defined

# Defining Features and Targets

In this cell, we define the features (columns) used for input to the model, which include `year`, `month`, `day`, and `country`. We also define the target variables for both subtasks, such as `hazard-category`, `product-category`, `hazard`, and `product`. These will be the labels we aim to predict.


In [14]:
# Define relevant features and targets
features = ['year', 'month', 'day', 'country']
targets_subtask1 = ['hazard-category','product-category']
targets_subtask2 = ['hazard','product']


# Label Encoding for Targets

Here, we encode the categorical target labels into numeric values using `LabelEncoder` from `sklearn`. This step is necessary for training the model, as models require numeric labels for classification tasks.
For each target, a new `LabelEncoder` is created, and the target column is transformed into numeric labels.


In [15]:
# Encode target labels to numeric values
label_encoders = {}
for target in targets_subtask1 + targets_subtask2:
    le = LabelEncoder()
    df[target] = le.fit_transform(df[target])
    label_encoders[target] = le


NameError: name 'df' is not defined

# Data Preparation for Training and Testing

This function splits the data into training and testing sets for each target variable. It also ensures that the features and corresponding targets are aligned and reset the indices for consistency. The function returns a dictionary containing the splits for each target.


In [16]:
# Prepare data for both title and text
def prepare_data(text_column):
    X = df[features + [text_column]]
    y_subtask1 = df[targets_subtask1]
    y_subtask2 = df[targets_subtask2]

    data_splits = {}
    for target in targets_subtask1 + targets_subtask2:
        X_train, X_test, y_train, y_test = train_test_split(
            X, df[target], test_size=0.2, random_state=42
        )

        # Reset indices to ensure matching
        X_train = X_train.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        X_test = X_test.reset_index(drop=True)
        y_test = y_test.reset_index(drop=True)

        data_splits[target] = (X_train, X_test, y_train, y_test)

    return data_splits


# Prepare Data for Title and Text Subtasks

This step prepares separate data splits for the `title` and `text` columns. It uses the `prepare_data` function to generate data splits for both input types, which will be used to train and evaluate the models.


In [18]:
# Prepare data for title and text
title_splits = prepare_data('title')
text_splits = prepare_data('text')


NameError: name 'df' is not defined

# Model Training and Evaluation

- This function trains and evaluates the model for each target variable. It creates the DataLoader objects for the training and testing data, sets up the model, and runs the training loop. It also evaluates the model's performance by calculating the F1 score and printing the classification report.

- The training loop updates the model's weights using backpropagation, and the evaluation phase computes the model's predictions and compares them with the true labels.



In [19]:
# Function to train and evaluate neural network models
def train_and_evaluate_nn(data_splits, targets, model_type='title'):
    f1_scores = []  # List to store F1 scores for each task

    for target in targets:
        print(f"\nStarting training for task: {target}")  # Print task message

        X_train, X_test, y_train, y_test = data_splits[target]

        # Prepare text data using the tokenizer
        if model_type == 'title':
            texts_train = X_train['title'].values
            texts_test = X_test['title'].values
        else:
            texts_train = X_train['text'].values
            texts_test = X_test['text'].values

        # Create DataLoader for training and testing
        train_dataset = TextDataset(texts_train, y_train, tokenizer, config['max_len'])
        test_dataset = TextDataset(texts_test, y_test, tokenizer, config['max_len'])

        train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)

        # Model setup
        num_labels = len(label_encoders[target].classes_)  # Corrected here
        model = AutoModelForSequenceClassification.from_pretrained(config['model_name'], num_labels=num_labels).to(device)

        optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
        criterion = nn.CrossEntropyLoss()

        # Training process
        model.train()
        for epoch in range(config['epochs']):
            print(f"Epoch {epoch+1}/{config['epochs']} - Training: {target}")
            progress_bar = tqdm(train_loader, desc=f"Training Epoch {epoch+1}", total=len(train_loader), leave=True)
            for batch in progress_bar:
                optimizer.zero_grad()
                input_ids = batch['input_ids'].squeeze(1).to(device)
                attention_mask = batch['attention_mask'].squeeze(1).to(device)
                labels = batch['label'].to(device)

                outputs = model(input_ids, attention_mask=attention_mask)
                loss = criterion(outputs.logits, labels)
                loss.backward()
                optimizer.step()
                progress_bar.set_postfix(loss=loss.item())

        # Evaluation process
        print(f"Evaluating model for task: {target}")
        model.eval()
        y_preds = []
        y_true = []
        with torch.no_grad():
            for batch in tqdm(test_loader, desc="Evaluating", total=len(test_loader), leave=True):
                input_ids = batch['input_ids'].squeeze(1).to(device)
                attention_mask = batch['attention_mask'].squeeze(1).to(device)
                labels = batch['label'].to(device)
                outputs = model(input_ids, attention_mask=attention_mask)
                _, preds = torch.max(outputs.logits, dim=1)
                y_preds.extend(preds.cpu().numpy())
                y_true.extend(labels.cpu().numpy())

        # Decode labels back to original categories using the label encoder
        decoded_preds = label_encoders[target].inverse_transform(y_preds)
        decoded_true = label_encoders[target].inverse_transform(y_true)

        # Calculate F1 score for the task
        f1 = f1_score(decoded_true, decoded_preds, average='weighted')
        f1_scores.append(f1)
        print(f"F1-Score for {target}: {f1}")

        # Print classification report
        print(f"Classification Report for {target}:\n")
        print(classification_report(decoded_true, decoded_preds,zero_division=0))

    return f1_scores  # Return the list of F1 scores for plotting


# Train and Evaluate for Title and Text Subtasks

This step trains and evaluates the model separately for the `title` and `text` features. It calls the `train_and_evaluate_nn` function for both types of input (title and text) and stores the F1 scores for comparison.


In [20]:
# Train and evaluate for both title and text
print("\nTraining and Evaluating for Title Tasks:")
title_f1_scores = train_and_evaluate_nn(title_splits, targets_subtask1 + targets_subtask2, model_type='title')

print("\nTraining and Evaluating for Text Tasks:")
text_f1_scores = train_and_evaluate_nn(text_splits, targets_subtask1 + targets_subtask2, model_type='text')



Training and Evaluating for Title Tasks:


NameError: name 'title_splits' is not defined

# Create DataFrames for F1 Scores

This cell creates two DataFrames, one for the title-focused F1 scores and another for the text-focused F1 scores. These DataFrames will be used for plotting the results.


In [21]:
# Create DataFrames for F1 scores for title and text
f1_scores_title_df = pd.DataFrame({
    'Task': targets_subtask1 + targets_subtask2,
    'F1-Score': title_f1_scores
})

f1_scores_text_df = pd.DataFrame({
    'Task': targets_subtask1 + targets_subtask2,
    'F1-Score': text_f1_scores
})


NameError: name 'title_f1_scores' is not defined

# Plot F1 Scores for Title and Text Subtasks

This cell visualizes the F1 scores for the title and text-based tasks by plotting them in a bar chart. It displays both sets of F1 scores in a combined chart for comparison.


In [22]:
# Plotting the data
plt.figure(figsize=(10, 6))

index_title = range(len(f1_scores_title_df))  # Position for Title-Focused
index_text = [i + len(f1_scores_title_df) for i in range(len(f1_scores_text_df))]  # Position for Text-Focused

# Plotting all Title-Focused F1-scores
plt.bar(index_title, f1_scores_title_df['F1-Score'], label='Title-Focused')

# Plotting all Text-Focused F1-scores (shifted on the x-axis after the Title-Focused bars)
plt.bar(index_text, f1_scores_text_df['F1-Score'], label='Text-Focused')

# Adding labels and title
plt.xlabel('Task')
plt.ylabel('F1-Score')
plt.title('F1-Scores for Title-Focused and Text-Focused Classification')

# Adjusting x-ticks to show all tasks
plt.xticks(range(len(f1_scores_title_df) + len(f1_scores_text_df)),
           list(f1_scores_title_df['Task']) + list(f1_scores_text_df['Task']),
           rotation=45)

# Adding legend
plt.legend()

# Displaying the plot
plt.tight_layout()
plt.show()


NameError: name 'f1_scores_title_df' is not defined

<Figure size 1000x600 with 0 Axes>